In [10]:
import definitions as df
import data_manipulation as dm
import sql_def as sql
import os
import numpy as np
import matplotlib.pyplot as plt
import plots
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import math
import torch.nn.functional as F

Grabing the data from SQL dataset

In [18]:
path_to_database = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project\data"
path_to_code = r"C:\Users\alito\Desktop\DeepLearning-Model\diff_transformers\project"
os.chdir(path_to_database)
track_from_sql =  sql.data_from_sql("database.db", "SELECT * FROM TRACKMATEDATA")
os.chdir(path_to_code)
track_with_frame = df.separate_trajectories(track_from_sql)
trajectories = df.separate_data(track_with_frame, False)
len(trajectories)

40252

Making Fake data

In [3]:
total_size = 100000
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 50

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)
trajectories

[[(734.0, 71.0),
  (732.309527, 67.374769),
  (730.619054, 63.749538),
  (728.928581, 60.124307),
  (727.238108, 56.499075),
  (725.547635, 52.873844),
  (723.857162, 49.248613),
  (722.166689, 45.623382),
  (720.476216, 41.998151),
  (718.785743, 38.37292),
  (717.09527, 34.747689),
  (715.404796, 31.122457),
  (713.714323, 27.497226),
  (712.02385, 23.871995),
  (710.333377, 20.246764),
  (708.642904, 16.621533),
  (706.952431, 12.996302),
  (705.261958, 9.37107),
  (703.571485, 5.745839),
  (701.881012, 2.120608),
  (700.190539, -1.504623),
  (698.500066, -5.129854),
  (696.809593, -8.755085),
  (695.11912, -12.380316),
  (693.428647, -16.005548),
  (691.738174, -19.630779),
  (690.047701, -23.25601),
  (688.357228, -26.881241),
  (686.666755, -30.506472),
  (684.976282, -34.131703),
  (683.285809, -37.756934),
  (681.595336, -41.382166),
  (679.904862, -45.007397),
  (678.214389, -48.632628),
  (676.523916, -52.257859),
  (674.833443, -55.88309),
  (673.14297, -59.508321),
  (671.4

In [19]:
norm_all_data = df.normalize_data(trajectories)
train, test = df.split_test_train(norm_all_data)
train_data, train_masked_point = df.mask_point_at_index(train, 2)
test_data, test_masked_point = df.mask_point_at_index(test, 2)

print(len(train_data))
print(len(test_masked_point))

36226
4026


In [20]:
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)

## DEBUG
print("src_data_tensor shape:", src_data_tensor.shape)
print("tgt_data_tensor shape:", tgt_data_tensor.shape)
print("src_masks_tensor shape:", src_masks_tensor.shape)

src_data_tensor shape: torch.Size([36226, 651, 2])
tgt_data_tensor shape: torch.Size([36226, 651, 2])
src_masks_tensor shape: torch.Size([36226, 651])


In [21]:
import model as md

input_dim = 2
hidden_dim = 64
output_dim = 2
rate = 0.0001

model = md.EnhancedTrajectoryModel(input_dim, hidden_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)


In [22]:
dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [23]:
for epoch in range(3):
        model.train()
        total_loss = 0

        for src, tgt, mask in dataloader:
            optimizer.zero_grad()
            output = model(src.view(-1, input_dim))
            output = output.view(src.shape)
            
            masked_output = output[mask]
            masked_tgt = tgt[mask]
            loss = loss_function(masked_output, masked_tgt)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 0.11136100723326785
Epoch 2, Loss: 0.08695088601149191
Epoch 3, Loss: 0.0858819267045894


In [24]:
t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
loss, accuracy = md.evaluate_model(model, t_src_data_tensor, t_tgt_data_tensor, t_src_masks_tensor)
print(f"Test Loss: {loss}")
print(f"Accuracy (within threshold): {accuracy}%")

Test Loss: 0.08358234167098999
Accuracy (within threshold): 2.558370590209961%
